Требуется классифицировать список абонентов телефонного оператора по переменной Churn. Набор данных содержит информацию о 3333 абонентах. Часть данных будет использоваться для обучения, часть для тестирования. Файл содержит 21 переменную для каждого абонента.
<table>
<thead>
<tr>
<th>Название</th>
<th>Описание</th>
</tr>
</thead>
<tbody>
<tr>
<td><strong>State</strong></td>
<td>Буквенный код штата</td>
</tr>
<tr>
<td><strong>Account length</strong></td>
<td>Как долго клиент обслуживается компанией</td>
</tr>
<tr>
<td><strong>Area code</strong></td>
<td>Префикс номера телефона</td>
</tr>
<tr>
<td><strong>Phone number</strong></td>
<td>Номер телефона</td>
</tr>
<tr>
<td><strong>International plan</strong></td>
<td>Международный роуминг (подключен/не подключен)</td>
</tr>
<tr>
<td><strong>Voice mail plan</strong></td>
<td>Голосовая почта (подключена/не подключена)</td>
</tr>
<tr>
<td><strong>Number vmail messages</strong></td>
<td>Количество голосовых сообщений</td>
</tr>
<tr>
<td><strong>Total day minutes</strong></td>
<td>Общая длительность разговоров днем</td>
</tr>
<tr>
<td><strong>Total day calls</strong></td>
<td>Общее количество звонков днем</td>
</tr>
<tr>
<td><strong>Total day charge</strong></td>
<td>Общая сумма оплаты за услуги днем</td>
</tr>
<tr>
<td><strong>Total eve minutes</strong></td>
<td>Общая длительность разговоров вечером</td>
</tr>
<tr>
<td><strong>Total eve calls</strong></td>
<td>Общее количество звонков вечером</td>
</tr>
<tr>
<td><strong>Total eve charge</strong></td>
<td>Общая сумма оплаты за услуги вечером</td>
</tr>
<tr>
<td><strong>Total night minutes</strong></td>
<td>Общая длительность разговоров ночью</td>
</tr>
<tr>
<td><strong>Total night calls</strong></td>
<td>Общее количество звонков ночью</td>
</tr>
<tr>
<td><strong>Total night charge</strong></td>
<td>Общая сумма оплаты за услуги ночью</td>
</tr>
<tr>
<td><strong>Total intl minutes</strong></td>
<td>Общая длительность международных разговоров</td>
</tr>
<tr>
<td><strong>Total intl calls</strong></td>
<td>Общее количество международных разговоров</td>
</tr>
<tr>
<td><strong>Total intl charge</strong></td>
<td>Общая сумма оплаты за международные разговоры</td>
</tr>
<tr>
<td><strong>Customer service calls</strong></td>
<td>Число обращений в сервисный центр</td>
</tr>
<tr>
<td><strong>Churn</strong></td>
<td>Признак оттока (потеря клиента/клиент пока остаётся)</td>
</tr>
</tbody>
</table>

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings(action="ignore")

Приведение данных к числовому виду и удаление ненужных при обучении данных

In [2]:
df = pd.read_csv('telecom_churn.csv')
df['international plan'] = pd.factorize(df['international plan'])[0]
df['voice mail plan'] = pd.factorize(df['voice mail plan'])[0]
df['churn'] = df['churn'].astype('int')
states = df['state']
y = df['churn']
df.drop(['state', 'churn', 'phone number', 'area code'], axis=1, inplace=True)

Создание дерева решений

In [3]:
X_train, X_test, y_train, y_test = train_test_split(df.values, y, test_size=0.3)
tree = DecisionTreeClassifier()
tree_params = {'max_depth': range(2, 15), 'max_features': range(1, 17)}
tree_grid = GridSearchCV(tree, tree_params, cv=5)
tree_grid.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=DecisionTreeClassifier(class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort=False, random_state=None,
                                              splitter='best'),
             iid='warn', n_jobs=None,
             param_grid={'max_depth': range(2, 15),
                         'max_features': range(1, 17)},
             pre_dispatch='2

Лучшие глубина дерева и количество признаков для данных условий

In [4]:
tree_grid.best_params_

{'max_depth': 6, 'max_features': 16}

In [5]:
tree_grid.best_score_

0.9455636519502786

In [13]:
accuracy_score(y_test, tree_grid.predict(X_test))

0.945

Визуализация дерева решений

In [26]:
export_graphviz(tree_grid.best_estimator_, feature_names=df.columns, out_file='churn_tree.gv', filled=True)
!dot -Tpng churn_tree.gv -o churn_tree.png

<img src="churn_tree.png">